In [1]:
import sys
import numpy as np
import gc
import matplotlib.pyplot as plt
import math

#sys.path.append("/home/nenciric/Documents/git/NMR_FF_tools/relaxation_times") 
import ExperimentalRelaxationTimes as ERT

gc.collect()


0

In [2]:
path="/home/nenciric/Documents/git/COR15A_Tobi/ExperimentalData/600MHz/Tobbi_COR15A_wt_0.9mM_20mM_Na-P_buffer_sample2_September2022/T1/experiment_110/"
field="600 MHz"
coeffs=[2,8,32,64]
outputs=[0,2,8,32,128]
author=["Ricky Nencini",  "ricky.nencini@helsinki.fi"]
info="Tobbi_COR15A_wt_0.9mM_20mM_Na-P_buffer_sample2_September2022"
increments=223
data_from_DynamicCenter="T1_223incr_no_0coeff_0output.txt"
data_from_DynamicCenter=path+data_from_DynamicCenter
results=[ERT.t1_t2_relaxations(data_from_DynamicCenter,field,0,0,author,info,increments)]
for coeff in coeffs:
    for outputP in outputs:
        data_from_DynamicCenter="T1_"+str(increments)+"incr_no_"+str(coeff)+"coeff_"+str(outputP)+"output.txt"
        data_from_DynamicCenter=path+data_from_DynamicCenter
        results.append(ERT.t1_t2_relaxations(data_from_DynamicCenter,field,coeff,outputP,author,info,increments))
        

In [6]:
results

[{'peaks': {'1': {'ppm': ['-97.274', '-218.151'],
    'experiment': array([4.24443536e+08, 4.25222697e+08, 4.15158102e+08, 3.76684597e+08,
           3.41511493e+08, 2.82319355e+08, 2.32653488e+08, 1.90636964e+08,
           1.10001542e+08, 7.77311220e+07]),
    'fit': (array([-0.9325281 , 19.92415721]),
     array([0.001129]),
     2,
     array([1.32134322, 0.50403581]),
     1.9984014443252818e-15),
    'points_used': 9},
   '2': {'ppm': ['-99.208', '-218.263'],
    'experiment': array([9948633., 9274010., 9305442., 6773313., 7538441., 5994742.,
           4972841., 5735706., 4800120.,       0.]),
    'fit': (array([-0.47875951, 16.02280489]),
     array([0.05545579]),
     2,
     array([1.31679107, 0.51581129]),
     1.3322676295501878e-15),
    'points_used': 6},
   '3': {'ppm': ['-99.724', '-217.682'],
    'experiment': array([1.19109539e+08, 1.30126346e+08, 1.22778990e+08, 1.03473009e+08,
           9.23236510e+07, 7.24841050e+07, 5.62079280e+07, 4.04031520e+07,
           1.99

In [3]:
ERT.compare_spectra(*results)

{'1': {'REFERENCE': {'ppm': ['-97.274', '-218.151'],
   'T1': 1.0722462943035191,
   'COEFFICIENTS': 64,
   'OUTPUT_POINTS': 0},
  'file0': {'ppm': ['-97.274', '-218.151'],
   'T1': 1.0723537489141635,
   'COEFFICIENTS': 0,
   'OUTPUT_POINTS': 0},
  'file1': {'ppm': ['-97.274', '-218.151'],
   'T1': 1.1063149820858869,
   'COEFFICIENTS': 2,
   'OUTPUT_POINTS': 0},
  'file2': {'ppm': ['-97.274', '-218.151'],
   'T1': 1.1063149820858869,
   'COEFFICIENTS': 2,
   'OUTPUT_POINTS': 2},
  'file3': {'ppm': ['-97.274', '-218.151'],
   'T1': 1.1063149820858869,
   'COEFFICIENTS': 2,
   'OUTPUT_POINTS': 8},
  'file4': {'ppm': ['-97.274', '-218.151'],
   'T1': 1.1063149820858869,
   'COEFFICIENTS': 2,
   'OUTPUT_POINTS': 32},
  'file5': {'ppm': ['-97.274', '-218.151'],
   'T1': 1.1063149820858869,
   'COEFFICIENTS': 2,
   'OUTPUT_POINTS': 128},
  'file6': {'ppm': ['-97.274', '-218.151'],
   'T1': 1.0483095858741815,
   'COEFFICIENTS': 8,
   'OUTPUT_POINTS': 0},
  'file7': {'ppm': ['-97.274', '-21

In [12]:

res1={'peaks': {'1': {'ppm': ['-97.274', '-218.151'],
    'experiment': np.array([4.24443536e+08, 4.25222697e+08, 4.15158102e+08, 3.76684597e+08,
           3.41511493e+08, 2.82319355e+08, 2.32653488e+08, 1.90636964e+08,
           1.10001542e+08, 7.77311220e+07]),
    'fit': (np.array([-0.9325281 , 19.92415721]),
     np.array([0.001129]),
     2,
     np.array([1.32134322, 0.50403581]),
     1.9984014443252818e-15),
    'points_used': 9},
   '2': {'ppm': ['-99.208', '-218.263'],
    'experiment': np.array([9948633., 9274010., 9305442., 6773313., 7538441., 5994742.,
           4972841., 5735706., 4800120.,       0.]),
    'fit': (np.array([-0.47875951, 16.02280489]),
     np.array([0.05545579]),
     2,
     np.array([1.31679107, 0.51581129]),
     1.3322676295501878e-15),
    'points_used': 6},
   '3': {'ppm': ['-99.724', '-217.682'],
    'experiment': np.array([1.19109539e+08, 1.30126346e+08, 1.22778990e+08, 1.03473009e+08,
           9.23236510e+07, 7.24841050e+07, 5.62079280e+07, 4.04031520e+07,
           1.99967050e+07, 1.15926370e+07]),
    'fit': (np.array([-1.30015424, 18.73736857]),
     np.array([0.00482653]),
     2,
     np.array([1.32134322, 0.50403581]),
     1.9984014443252818e-15),
    'points_used': 9}},
  'mixing_times': np.array([0.02, 0.05, 0.1 , 0.2 , 0.3 , 0.5 , 0.7 , 0.9 , 1.5 , 1.9 ]),
  'INFO': {'ANALYZED': '2022-09-16',
   'AUTHOR': ['Ricky Nencini', 'ricky.nencini@helsinki.fi'],
   'FIELD': '600 MHz',
   'COEFFICIENTS': 0,
   'OUTPUT_POINTS': 0,
   'INFO': 'Tobbi_COR15A_wt_0.9mM_20mM_Na-P_buffer_sample2_September2022'}}

res2={'peaks': {'62': {'ppm': ['-116.161', '-217.635'],
    'experiment': np.array([66478192., 72501987., 69598778., 60744690., 53275459., 42383649.,
           32022058., 25730733., 11000950.,  6954201.]),
    'fit': (np.array([-1.28486478, 18.1831368 ]),
     np.array([0.0042219]),
     2,
     np.array([1.32134322, 0.50403581]),
     1.9984014443252818e-15),
    'points_used': 9},
   '63': {'ppm': ['-116.323', '-217.553'],
    'experiment': np.array([1.15089346e+08, 1.23913842e+08, 1.15998009e+08, 9.65010190e+07,
           8.24260590e+07, 6.12855820e+07, 4.70384630e+07, 3.39541190e+07,
           1.56976280e+07, 8.68386600e+06]),
    'fit': (np.array([-1.42485561, 18.67372985]),
     np.array([0.00763195]),
     2,
     np.array([1.32134322, 0.50403581]),
     1.9984014443252818e-15),
    'points_used': 9},
   '64': {'ppm': ['-116.903', '-217.711'],
    'experiment': np.array([9.10002010e+07, 1.00430069e+08, 9.31383440e+07, 8.10552450e+07,
           6.88609240e+07, 5.41042920e+07, 4.03631270e+07, 3.08659750e+07,
           1.49261290e+07, 9.14037500e+06]),
    'fit': (np.array([-1.29495653, 18.45651605]),
     np.array([0.00670722]),
     2,
     np.array([1.32134322, 0.50403581]),
     1.9984014443252818e-15),
    'points_used': 9},
   '65': {'ppm': ['-116.999', '-217.582'],
    'experiment': np.array([70887359., 76480352., 72474143., 63479020., 56439975., 45805290.,
           35105790., 27539194., 13967531.,  8254544.]),
    'fit': (np.array([-1.19390693, 18.21468319]),
     np.array([0.00197962]),
     2,
     np.array([1.32134322, 0.50403581]),
     1.9984014443252818e-15),
    'points_used': 9},
   '66': {'ppm': ['-117.708', '-217.658'],
    'experiment': np.array([43693176., 46062435., 45427349., 39815536., 35938034., 28993852.,
           22526611., 16124704.,  8836715.,  7579848.]),
    'fit': (np.array([-1.05213674, 17.68766038]),
     np.array([0.0620024]),
     2,
     np.array([1.32134322, 0.50403581]),
     1.9984014443252818e-15),
    'points_used': 9}},
  'mixing_times': np.array([0.02, 0.05, 0.1 , 0.2 , 0.3 , 0.5 , 0.7 , 0.9 , 1.5 , 1.9 ]),
  'INFO': {'ANALYZED': '2022-09-16',
   'AUTHOR': ['Ricky Nencini', 'ricky.nencini@helsinki.fi'],
   'FIELD': '600 MHz',
   'COEFFICIENTS': 2,
   'OUTPUT_POINTS': 32,
   'INFO': 'Tobbi_COR15A_wt_0.9mM_20mM_Na-P_buffer_sample2_September2022'}}

jeKrasneNaSvete=[res1,res2]

In [18]:
compare_spectra(*jeKrasneNaSvete)

['-97.274', '-218.151']
['-99.208', '-218.263']
['-99.724', '-217.682']
[[18.89404734 16.9646277  16.4370672 ]
 [19.05838411 17.12972052 16.59950126]
 [19.63393086 17.7036078  17.17902448]
 [19.73320516 17.80402881 17.27528943]
 [20.43994631 18.50988992 17.98401601]]


{'62': {'REFERENCE': {'ppm': ['-116.161', '-217.635'],
   'T1': 0.7782920160672472,
   'COEFFICIENTS': 2,
   'OUTPUT_POINTS': 32},
  'file0': {'ppm': ['-99.724', '-217.682'],
   'T1': 0.7691395137856876,
   'COEFFICIENTS': 0,
   'OUTPUT_POINTS': 0}},
 '63': {'REFERENCE': {'ppm': ['-116.323', '-217.553'],
   'T1': 0.7018254993570893,
   'COEFFICIENTS': 2,
   'OUTPUT_POINTS': 32}},
 '64': {'REFERENCE': {'ppm': ['-116.903', '-217.711'],
   'T1': 0.7722266939724995,
   'COEFFICIENTS': 2,
   'OUTPUT_POINTS': 32}},
 '65': {'REFERENCE': {'ppm': ['-116.999', '-217.582'],
   'T1': 0.8375862262563465,
   'COEFFICIENTS': 2,
   'OUTPUT_POINTS': 32}},
 '66': {'REFERENCE': {'ppm': ['-117.708', '-217.658'],
   'T1': 0.9504468021903694,
   'COEFFICIENTS': 2,
   'OUTPUT_POINTS': 32}}}

In [17]:
def compare_spectra(*files):
    most_peaks=0
    for i,file in enumerate(files):
        if len(file['peaks'])>most_peaks:
            most_peaks=len(file['peaks'])
            position=i
            
    reference=files[position]
    other_files=[]
    
    for i,file in enumerate(files):
        if not i==position:
            other_files.append(file)
    
    files=other_files
    
    relaxation_times={}
    for ref_peak in reference['peaks']:
        relaxation_times[ref_peak]={}
        relaxation_times[ref_peak]["REFERENCE"]={}
        relaxation_times[ref_peak]["REFERENCE"]["ppm"]=reference['peaks'][ref_peak]["ppm"]
        relaxation_times[ref_peak]["REFERENCE"]["T1"]=  -1/reference['peaks'][ref_peak]["fit"][0][0]
        relaxation_times[ref_peak]["REFERENCE"]["COEFFICIENTS"]=  reference["INFO"]["COEFFICIENTS"]
        relaxation_times[ref_peak]["REFERENCE"]["OUTPUT_POINTS"]=  reference["INFO"]["OUTPUT_POINTS"]
        #relaxation_times[ref_peak]["REFERENCE"]["error"]=  -1/reference[ref_peak]["fit"][2][0]
        
    for k,file in enumerate(files):
        distances=np.zeros([len(reference['peaks']),len(file['peaks'])])
        for i,ref_peak in enumerate(reference['peaks']):
            #print(float(reference[ref_peak]["ppm"][0]))
            for j,file_peak in enumerate(file['peaks']):
                distances[i,j]=(math.dist([float(reference['peaks'][ref_peak]["ppm"][0]), 
                                           float(reference['peaks'][ref_peak]["ppm"][1])],
                                           [float(file['peaks'][file_peak]["ppm"][0]),
                                            float(file['peaks'][file_peak]["ppm"][1])
                                           ])) 
        
        
        if len(reference['peaks'])>len(file['peaks']) or len(reference['peaks'])==len(file['peaks']):
            reference_keys=list(reference['peaks'])
            for i,file_peak in enumerate(file['peaks']):
                print(file['peaks'][file_peak]["ppm"])
                key=np.where(distances[:,i]==min(distances[:,i]))[0][0]
                relaxation_times[reference_keys[key]]["file"+str(k)]={}
                relaxation_times[reference_keys[key]]["file"+str(k)]["ppm"]=file['peaks'][file_peak]["ppm"]
                relaxation_times[reference_keys[key]]["file"+str(k)]["T1"]=-1/file['peaks'][file_peak]["fit"][0][0]
                relaxation_times[reference_keys[key]]["file"+str(k)]["COEFFICIENTS"]=  file["INFO"]["COEFFICIENTS"]
                relaxation_times[reference_keys[key]]["file"+str(k)]["OUTPUT_POINTS"]=  file["INFO"]["OUTPUT_POINTS"]
                
        if len(reference['peaks'])<len(file['peaks']):  
            file_keys=list(file['peaks'])
            for i,ref_peak in enumerate(reference['peaks']):
                key=np.where(distances[i,:]==min(distances[i,:]))[0][0]
                relaxation_times[ref_peak]["file"+str(k)]={}
                relaxation_times[ref_peak]["file"+str(k)]["ppm"]=file['peaks'][file_keys[key]]["ppm"]
                relaxation_times[ref_peak]["file"+str(k)]["T1"]=-1/file['peaks'][file_keys[key]]["fit"][0][0]
                relaxation_times[ref_peak]["file"+str(k)]["COEFFICIENTS"]=  file["INFO"]["COEFFICIENTS"]
                relaxation_times[ref_peak]["file"+str(k)]["OUTPUT_POINTS"]=  file["INFO"]["OUTPUT_POINTS"]
                
    print(distances)
            
    
    return relaxation_times

In [4]:
x_axis=[]
y_axis=[]
for peak in results:
    x_axis.append(int(peak))
    one_line=[]
    for i,spectra in enumerate(results[peak]):
        one_line.append(results[peak][spectra]["T1"])
    y_axis.append(one_line)

#y_axis=np.reshape(y_axis, (len(y_axis), i+1)).T

[{'peaks': {'1': {'ppm': ['-97.274', '-218.151'],
    'experiment': array([4.24443536e+08, 4.25222697e+08, 4.15158102e+08, 3.76684597e+08,
           3.41511493e+08, 2.82319355e+08, 2.32653488e+08, 1.90636964e+08,
           1.10001542e+08, 7.77311220e+07]),
    'fit': (array([-0.9325281 , 19.92415721]),
     array([0.001129]),
     2,
     array([1.32134322, 0.50403581]),
     1.9984014443252818e-15),
    'points_used': 9},
   '2': {'ppm': ['-99.208', '-218.263'],
    'experiment': array([9948633., 9274010., 9305442., 6773313., 7538441., 5994742.,
           4972841., 5735706., 4800120.,       0.]),
    'fit': (array([-0.47875951, 16.02280489]),
     array([0.05545579]),
     2,
     array([1.31679107, 0.51581129]),
     1.3322676295501878e-15),
    'points_used': 6},
   '3': {'ppm': ['-99.724', '-217.682'],
    'experiment': array([1.19109539e+08, 1.30126346e+08, 1.22778990e+08, 1.03473009e+08,
           9.23236510e+07, 7.24841050e+07, 5.62079280e+07, 4.04031520e+07,
           1.99

In [6]:
daticka=["lelky","deti","achjo","tenPodzimJeKrasny"]

In [6]:
from datetime import date
today = date.today()
print(today)

2022-09-16


In [14]:
pokus(*daticka)

lelky
deti
achjo
tenPodzimJeKrasny


In [ ]:
no_LP=ERT.t1_t2_relaxations("T1_227incr_no_0coeff_0output.txt")[0]
no_16_2=ERT.t1_t2_relaxations("T1_227incr_no_16coeff_2output.txt")[0]
no_16_8=ERT.t1_t2_relaxations("T1_227incr_no_16coeff_8output.txt")[0]
no_16_32=ERT.t1_t2_relaxations("T1_227incr_no_16coeff_32output.txt")[0]
no_16_128=ERT.t1_t2_relaxations("T1_227incr_no_16coeff_128output.txt")[0]

no_32_2=ERT.t1_t2_relaxations("T1_227incr_no_32coeff_2output.txt")[0]
no_32_8=ERT.t1_t2_relaxations("T1_227incr_no_32coeff_8output.txt")[0]
no_32_32=ERT.t1_t2_relaxations("T1_227incr_no_32coeff_32output.txt")[0]
no_32_128=ERT.t1_t2_relaxations("T1_227incr_no_32coeff_128output.txt")[0]

no_64_2=ERT.t1_t2_relaxations("T1_227incr_no_64coeff_2output.txt")[0]
no_64_8=ERT.t1_t2_relaxations("T1_227incr_no_64coeff_8output.txt")[0]
no_64_32=ERT.t1_t2_relaxations("T1_227incr_no_64coeff_32output.txt")[0]
no_64_128=ERT.t1_t2_relaxations("T1_227incr_no_64coeff_128output.txt")[0]